# PGA DYN - Double Hooke

This notebook implements the final example from the ["May The Forque Be With You"](https://enki.ws/ganja.js/examples/pga_dyn.html) implementation supplement to [PGAdyn](https://bivector.net/PGADYN.html).

In this example we will dangle a block from two different strings.
The main purpose of this example is to demonstrate how to implement a non-trivial inertia tensors in `kingdon`. For more detail on PGA itself we still highly recommend the recources linked above. The `ganja.js` code should be legible for an experienced `kingdon` user.

In [1]:
%pip install -q kingdon anywidget==0.9.9 ipywidgets==8.1.3

Note: you may need to restart the kernel to use updated packages.


In [2]:
from kingdon import Algebra

alg = Algebra(3, 0, 1)
globals().update(alg.blades)

As an integrator we will use the Runge–Kutta method:

In [3]:
def RK4(f, y, h):
    k1 = f(*y)
    k2 = f(*[yi + 0.5*h*k1i for yi, k1i in zip(y, k1)])
    k3 = f(*[yi + 0.5*h*k2i for yi, k2i in zip(y, k2)])
    k4 = f(*[yi + h*k3i for yi, k3i in zip(y, k3)])
    return [yi + (h/3)*(k2i + k3i + (k1i + k4i)*0.5) 
            for yi, k1i, k2i, k3i, k4i in zip(y, k1, k2, k3, k4)]

Then we have to define a box:

In [4]:
d = 3
size = [0.2, 1, 0.2]
vertexes = [
    alg.vector([1, *[s*(((i>>j)%2)-0.5) for j, s in enumerate(size)]]).dual()
    for i in range(2**d)
]
faces = [
    0xCC00FF,[0,1,2],[1,2,3],
    0x0400ff,[4,5,6],[5,6,7],
    0xfbff00,[0,1,4],[4,5,1],
    0xffa200,[2,6,7],[2,3,7],
    0x44ff00,[0,4,2],[4,2,6],
    0x00aaff,[1,5,7],[1,3,7]
]
faces = [
    [vertexes[i] for i in x] if isinstance(x, list) else x
     for x in faces
]

The inertia tensor in the body frame can be encoded in a single bivector $I$:

In [5]:
mass = 1
I = 1/12*mass*( 
    (size[1]**2+size[2]**2)*e01 + 
    (size[2]**2+size[0]**2)*e02 + 
    (size[0]**2+size[1]**2)*e03 + 
    12*e12 + 12*e13 + 12*e23
)
I

0.0867 𝐞₀₁ + 0.00667 𝐞₀₂ + 0.0867 𝐞₀₃ + 1.0 𝐞₁₂ + 1.0 𝐞₁₃ + 1.0 𝐞₂₃

The (inverse) inertia map can be applied to the rate bivector $B$ in the body frame.

In [6]:
A  = lambda B: B.dual().map(lambda k, v: v * getattr(I, alg.bin2canon[k]))
Ai = lambda B: B.map(lambda k, v: v / getattr(I, alg.bin2canon[k])).undual()

We can now define the two attachement points and describe how the motor $M$ and rate bivector $B$ are updated by the integrator.

In [7]:
attach  = vertexes[5] + e2.dual()
attach2 = vertexes[1] + e2.dual() + 0.5*e1.dual()

# Uncomment the line @alg.register(symbolic=True) to make the forque computation ~25x faster.
# However this breaks animation interactivity because attach & attach2 are not variables of the function.
# It is good to be aware that such a significant speed-up is available however.
# @alg.register(symbolic=True)
def forques(M, B):
    Gravity = (~M >> -9.81*e02).dual()
    Damping = -0.25 * B.grade(2).dual()
    Hooke = -8*(~M >> attach) & vertexes[5]
    Hooke2 = -8*(~M >> attach2) & vertexes[1]
    return (Gravity + Hooke + Hooke2 + Damping).grade(2)  # Ensure a pure bivector because in kingdon<=1.1.0, >> doesn't. Maybe this will change in the future.

# Change in M and B
dState = lambda M, B: [
    -0.5 * M * B,
    Ai(forques(M, B) - A(B).cp(B))
]

Finally we are ready to create an animation of a block on two springs.

In [8]:
state  = [1-0.5*e02, 38.35*e13 + 35.27*e12 - 5*e01]

def graph_func():
    global state
    
    state = RK4(dState, state, 0.01)
    new_faces = [
        [state[0] >> point for point in x] if isinstance(x, list) else x
         for x in faces
    ]

    return [
        0xCC00FF,
        *new_faces,
        0,
        attach, 
        attach2,
        [attach, state[0] >> vertexes[5]],
        [attach2, state[0] >> vertexes[1]],
    ]

alg.graph(
    graph_func,
    animate=True,
    lineWidth=2,
    pointRadius=2,
    alpha=1,
    gl=1,
    height='800px',
)

GraphWidget(cayley=[['1', 'e0', 'e1', 'e2', 'e3', 'e01', 'e02', 'e03', 'e12', 'e13', 'e23', 'e012', 'e013', 'e…